In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
new_model = tf.keras.models.load_model('UndersampledModel5\\tf_cnnmodel')
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1210, 32)          640032    
_________________________________________________________________
conv1d (Conv1D)              (None, 1209, 32)          2080      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 643,234
Trainable params: 643,234
Non-trainable params: 0
__________________________________________________

In [3]:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [4]:
test_data = pd.read_csv('nusadua_test.csv', converters={'reviews' : str})
# test_data.drop(["Date", "Content"], axis=1, inplace=True)
# test_data.fillna(0)
# test_data = test_data.fillna("")
test_data.sample(5)

,Date,Content,review,scores,compound,sentiment
666,Jun-16,The beach very nice with a very beautiful view...,nice beautiful view alot rubish waterbut well ...,"{'neg': 0.0, 'neu': 0.362, 'pos': 0.638, 'comp...",0.8316,1
391,Dec-16,Stunning! We stayed at Holiday Inn and wandere...,stun stay holiday inn wander along buy corn sa...,"{'neg': 0.0, 'neu': 0.538, 'pos': 0.462, 'comp...",0.9001,1
344,Jul-16,"When it is low tide (most of the day), the bea...",low tide day disgust shallow reef barrier cant...,"{'neg': 0.169, 'neu': 0.681, 'pos': 0.15, 'com...",-0.0772,0
645,Jan-17,"Great place, great people, perfect relax. We w...",great place great people perfect relax two day...,"{'neg': 0.0, 'neu': 0.486, 'pos': 0.514, 'comp...",0.9413,1
521,Sep-17,"The best holidayss ever...romántic, amazing pe...",best holiday everromántic amazing people cultu...,"{'neg': 0.0, 'neu': 0.238, 'pos': 0.762, 'comp...",0.9595,1


In [13]:
try :
    x_test  = np.array( tokenizer.texts_to_sequences(test_data['review'].tolist()) )
    # x_test = tokenizer
    x_test = pad_sequences(x_test, padding='post', maxlen=1210)
except AttributeError :
    pass


In [12]:
# Generate predictions (probabilities -- the output of the last layer)
# on test  data using `predict`
print("Generate predictions for all samples")
predictions = new_model.predict(x_test)
print(predictions)
predict_results = predictions.argmax(axis=1)

Generate predictions for all samples


NameError: name 'x_test' is not defined

In [ ]:
test_data['pred_sentiment'] = predict_results
test_data['pred_sentiment'] = np.where((test_data.pred_sentiment == 0),'0',test_data.pred_sentiment)
test_data['pred_sentiment'] = np.where((test_data.pred_sentiment == '1'),'1',test_data.pred_sentiment)

In [ ]:
labels = ['0', '1']
    
print(classification_report(test_data['sentiment'].values.astype(int).astype(str),test_data['pred_sentiment'].values,labels=labels))

In [ ]:
confusion_matrix = pd.crosstab(test_data['sentiment'], test_data['pred_sentiment'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

In [ ]:
roc_auc = roc_auc_score(test_data['sentiment'], test_data['pred_sentiment'])

print("ROC AUC Score")
print(roc_auc)

In [ ]:
#Test Sentimen
sequence20 = tokenizer.texts_to_sequences(['perfect morning walk amazing sunrise'])


test20 = pad_sequences(sequence20, padding="post")
labels[np.around(new_model.predict(test20)).argmax(axis=1)[0]]

In [ ]:
sequence30 = tokenizer.texts_to_sequences(['dont recommend rubbish everywhere'])


test30 = pad_sequences(sequence30, padding="post")
labels[np.around(new_model.predict(test30)).argmax(axis=1)[0]]